In [2]:
import numpy as np
import  h5py
import matplotlib.pyplot as plt
import time
import rasterio
from rasterio.windows import Window

In [3]:
file_name = r'H:/lst-national-mosaic-july2024.hdf5'


In [ ]:
# print the name of levels and their shapdes
with h5py.File(file_name, 'r') as f:
    for item in f:
        print(item)
        print(f[item].shape)

In [5]:
rasterAddress = r'G:/NLCD/nlcd_2021_land_cover_l48_20230630.img'
with rasterio.open(rasterAddress,'r') as rst:
    metadata = rst.profile
    rstShape = rst.shape
metadata

{'driver': 'HFA', 'dtype': 'uint8', 'nodata': None, 'width': 161190, 'height': 104424, 'count': 1, 'crs': CRS.from_wkt('PROJCS["Albers_Conical_Equal_Area",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["meters",1],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'), 'transform': Affine(30.0, 0.0, -2493045.0,
       0.0, -30.0, 3310005.0), 'blockxsize': 512, 'blockysize': 512, 'tiled': True}

In [6]:
output_file = 'outputRaster.img'
metadata['count']= 1
metadata['dtype']='int16'
metadata['nodata']= 9999
metadata

{'driver': 'HFA', 'dtype': 'int16', 'nodata': 9999, 'width': 161190, 'height': 104424, 'count': 1, 'crs': CRS.from_wkt('PROJCS["Albers_Conical_Equal_Area",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["meters",1],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'), 'transform': Affine(30.0, 0.0, -2493045.0,
       0.0, -30.0, 3310005.0), 'blockxsize': 512, 'blockysize': 512, 'tiled': True}

In [ ]:
start_time = time.time()
lstError =[]
window_size = 5000
with rasterio.open(output_file, 'w', **metadata) as dst:
    for i in range(0, rstShape[0], window_size):
        for j in range(0, rstShape[1], window_size):
            end_i = min(i + window_size, rstShape[0])
            end_j = min(j + window_size, rstShape[1])
            # if (i >=30000) and (i <=40000) and (j>=100000) and (j<=110000): # a test
            print(i,end_i,j,end_j)
            arrays = []
            with h5py.File(file_name, 'r') as f:
                for l in range(1, 53):
                    level_name = f'level{l:02d}'
                    if level_name in f:
                        array_2d = f[level_name][i:end_i, j:end_j]  # Read the first 5 rows and all columns
                        arrays.append(array_2d)  # Append the 2D array to the list
            # print('yes')
            
            # Stack all 2D arrays into a 3D NumPy array
            stacked_array = np.stack(arrays)
            stacked_array_with_nan = np.where(stacked_array == 0, np.nan, stacked_array)
            # convert the DN to T and then C.
            # a = ((stacked_array_with_nan*0.00341802)+149.0)- 273.15
            a = ((stacked_array_with_nan*0.00341802)-124.149999) # 149 -273.15= - 124.14999999
            s = (np.nanmedian(a,axis=0))
            s = np.nan_to_num(s, nan=99.99)
            v = (s*100).astype('int16')
            # Write the window to the raster file
            win = Window.from_slices((i, end_i), (j, end_j))
            # dst.write(v,window=win, indexes=1)
            # del arrays,a,s,v,stacked_array,stacked_array_with_nan

            # except:
            #     lstError.append([i,j])
            #     print('got an erro')
            break   
endtime = time.time()
print('time is: ',(endtime - start_time)/60)

In [ ]:
55/9